In [ ]:
# Image fusion Using Laplacian Pyramid

# ImgFused = FusImgsMultiDepths(ImgListFile , 4)
# ImgListFile: a (7*height*height) list file that includes the loaded images of the 7 depths


def cust_laplacian_fuse(M1, M2, zt, ap, mp):
    
    M1 = M1.astype(float)
    M2 = M2.astype(float)
    
    if M1.shape != M2.shape:
        raise ValueError('Input images are not of same size')

    w = np.array([1, 4, 6, 4, 1]) / 16
    w = w.reshape((1, 5))  # Ensure w is 2D

    E = [None] * zt
    zl = []
    sl = []

    for i1 in range(zt):
        z, s = M1.shape
        zl.append(z)
        sl.append(s)
        
        ew = [0, 0]
        if z % 2 != 0:
            ew[0] = 1
        if s % 2 != 0:
            ew[1] = 1
        
        if any(ew):
            M1 = adb(M1, ew)
            M2 = adb(M2, ew)
        
        G1 = convolve2d(convolve2d(es2(M1, 2), w, mode='valid'), w.T, mode='valid')
        G2 = convolve2d(convolve2d(es2(M2, 2), w, mode='valid'), w.T, mode='valid')
    
        M1T = convolve2d(convolve2d(es2(undec2(dec2(G1)), 2), 2 * w, mode='valid'), (2 * w).T, mode='valid')
        M2T = convolve2d(convolve2d(es2(undec2(dec2(G2)), 2), 2 * w, mode='valid'), (2 * w).T, mode='valid')
        
        E[i1] = select_cval_fun(M1 - M1T, M2 - M2T, ap)
    
        M1 = dec2(G1)
        M2 = dec2(G2)
    

    
    for i1 in range(zt-1, -1, -1):
        M1T = convolve2d(convolve2d(es2(undec2(M1), 2), 2 * w, mode='valid'), (2 * w).T, mode='valid')
        M1 = M1T + E[i1]
        M1 = M1[:zl[i1], :sl[i1]]

    return M1


def select_cval_fun(M1, M2, ap):
    
    if M1.shape != M2.shape:
        raise ValueError('Input images are not of same size')
    
    if ap[0] == 1:
        mm = np.abs(M1) > np.abs(M2)
        Y = mm * M1 + (~mm) * M2
    elif ap[0] == 2:
        um = 3
        th = 0.75
        S1 = convolve2d(es2(M1 * M1, um // 2), np.ones((um, um)), mode='valid')
        S2 = convolve2d(es2(M2 * M2, um // 2), np.ones((um, um)), mode='valid')
        MA = convolve2d(es2(M1 * M2, um // 2), np.ones((um, um)), mode='valid')
        MA = 2 * MA / (S1 + S2 + np.finfo(float).eps)
        m1 = MA > th
        m2 = S1 > S2
        w1 = 0.5 - 0.5 * (1 - MA) / (1 - th)
        Y = (~m1) * ((m2 * M1) + ((~m2) * M2))
        Y = Y + (m1 * ((m2 * M1 * (1 - w1)) + ((m2) * M2 * w1) + ((~m2) * M2 * (1 - w1)) + ((~m2) * M1 * w1)))
    elif ap[0] == 3:
        um = 3
        A1 = scipy.ndimage.maximum_filter(np.abs(es2(M1, um // 2)), size=(um, um))
        A2 = scipy.ndimage.maximum_filter(np.abs(es2(M2, um // 2)), size=(um, um))
        mm = (convolve2d((A1 > A2).astype(float), np.ones((um, um)), mode='valid')) > (um * um / 2)
        Y = mm * M1 + (~mm) * M2
    elif ap[0] == 4:
        mm = M1 > M2
        Y = mm * M1 + (~mm) * M2
    else:
        raise ValueError('unknown option')
        
    return Y


def select_bval_fun(M1, M2, mp):
    if mp == 1:
        return M1
    elif mp == 2:
        return M2
    elif mp == 3:
        return (M1 + M2) / 2
    else:
        raise ValueError('unknown option')

def es2(X, n):
    return np.pad(X, n, mode='edge')

def dec2(X):
    return X[::2, ::2]

def undec2(X):
    z, s = X.shape
    Y = np.zeros((2 * z, 2 * s))
    Y[::2, ::2] = X
    return Y

def adb(X, bd):
    z, s = X.shape
    Y = np.zeros((z + bd[0], s + bd[1]))
    Y[:z, :s] = X
    if bd[0] > 0:
        Y[z:, :s] = X[z-1::-1, :s]
    if bd[1] > 0:
        Y[:z, s:] = X[:, s-1::-1]
    if bd[0] > 0 and bd[1] > 0:
        Y[z:, s:] = X[z-1::-1, s-1::-1]
    return Y


# -----------------------------------------------------

def FusImgsMultiDepths(ImgFilesList , fuse_lev):
    
    M1 = ImgFilesList[0]; 
    M1 = resize(M1, (512, 512))
    A = M1.astype(np.float64)

    fused_imgs = A

    for i in range(1,7):
    
        M2 = M1 = ImgFilesList[i]
        M2 = resize(M2, (512, 512))
        B = M2.astype(np.float64)
        fused_imgs = cust_laplacian_fuse(fused_imgs, B, fuse_lev, [3], 3)

    F1 = fused_imgs
    image_float = F1.astype(np.float64)
    alpha = 255  # Contrast control
    beta = 0   # Brightness control

    adjusted = cv2.convertScaleAbs(image_float, alpha=alpha, beta=beta)

    return adjusted

# -----------------------------------------------------